## Remove prompt pulse Demo
<div class="alert alert-block alert-info">
    
<p> <b>Information:</b> This notebook shows a developed <code>scipp</code> function for "removing prompt pulses" and compares its output to those from Mantid's original algorithm.</p>
    
<p><b>Requirements:</b> To run this notebook, you need <code>mantid</code>, <code>scipp</code> and <code>numpy</code> installed as well as the Python script <code>remove_prompt_pulses.py</code> placed in the same folder as this notebook.</p>
    
<p>The required datafile is available from Mantid: <a href="http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e">PG3_4844_event.nxs</a>. Once downloaded the file should be renamed and placed in the same folder as this notebook.</p>
</div>

In [ ]:
import mantid.simpleapi as mapi
import numpy as np
import scipp as sc
import scippneutron as scn

import remove_prompt_pulses

input_file = 'PG3_4844_event.nxs'

### Mantid
#### Load datafile and apply `RemovePromptPulse`
With `width=50`, counts with `16667 <= time-of-flight <= 16717 microseconds` should be masked. This time-of-flight range has been checked in Mantid's log when running the following two algorithms in MantidWorkbench.

In [ ]:
ws = mapi.Load(input_file)
wsout = mapi.RemovePromptPulse(ws, width=50)

In [ ]:
nb_hist = ws.getNumberHistograms()
nb_hist

#### Check the number of events removed

In [ ]:
total_events_removed = 0

for i in range(nb_hist):
    evList = ws.getSpectrum(i)
    nb_events = evList.getNumberEvents()
    if nb_events != 0:
        total_events_removed += (nb_events - wsout.getSpectrum(i).getNumberEvents())

print(f"{total_events_removed} events have been removed after applying `RemovePromptPulse`.")

### Scipp

Since the behaviour of `RemovePromptPulse` in `Mantid` is not fully understood (see comment above), the output of `remove_prompt_pulse` will be checked "by hand".

In [ ]:
# Loading data and applying `remove_prompt_pulses`
sample = scn.load(input_file)
sample_out = remove_prompt_pulses.remove_prompt_pulses(sample, 50)

The range of tof values to mask is the same as in Mantid. The output dataset has a new  mask `prompt-pulse-bin`.

In [ ]:
sample_out

#### Check number of events removed

In [ ]:
counts_masked_dataset = sc.sum(sample_out.bins.size()).data.value
counts_ini_dataset = sc.sum(sample.bins.size()).data.value

print(f"{counts_ini_dataset-counts_masked_dataset} events have been removed by remove_prompt_pulse")

### Conclusion

The number of masked events is the same in scipp and mantid.